# Spreadsheet to Database

This tutorial is targeted at Data Engineering beginners.  In this lab we will upgrade our inventory management system from a spreadsheet into a database.

### What is a Database

A _database_, at its core, is an organized collection of data. Databases are meant to be fast at retrieving, storing, and updating these collections. To help keep things organized, a database keeps related data in a collection called a _table_. Data is stored inside a table as _columns_ and _rows_. The _columns_ identify the data that is being stored in the table, and the _rows_ are the actual data itself. Refer to the figure below for a basic example of organized data inside the database, with rows and columns.

| name | city | code |
|-------|-------|-------|
| Detroit Metropolitan Airport | Romulous | DTW |
| Seattle-Tacoma International Airport | SeaTac | SEA |
| Los Angeles International Airport | Los Angeles | LAX |

### Moving Inventory Data from a Spreadsheet into a Database

#### Create the Database
In this lab we will be importing our sample inventory data from a CSV into a SQL Database. First in Amazon lets create a database.

1) [Navigate to Amazon RDS](https://console.aws.amazon.com/rds/home?region=us-west-2#databases:)

2) Select Create database

3) Select Engine Type: `MySql`, version: choose the latest `MySQL 8.0.xx` version, templates: `Free Tier`
![MySql Config](./assets/lab1/database_configure.png)
![Template](./assets/lab1/db_template.png)

4) Give the database a name, for example: `InventoryDB` and a password, such as `demotest123`.  
![MySql Config pt2](./assets/lab1/database_configure_2.png)

5) Ensure the database is publicly accessible under the `Additional connectivity configuration` and add the `cloudacademylabs-MySQLSecurityGroup-...` security group, so that SageMaker can connect.
![MySql Config pt3](./assets/lab1/database_configure_public.png)

6) Leave the rest of the settings the same and click `Create Database`

Sit back and wait for Amazon to create the database. It usually completes in under 10 minutes.

#### Ensure We Can Connect From SageMaker and Create Database
    

Click your database name to view it's details. Under the connectivity and security portion, find the endpoint and copy it into the code below, followed by the password you set for the RDS database.
![database hostname](./assets/lab1/database_hostname.png)

In [22]:
! pip install mysql-connector

import mysql.connector
## Make sure the Host matches your RDS instance along with the username/password created earlier.
host = "inventorydb.cmzr2pvb7lzt.us-west-2.rds.amazonaws.com"
user = "admin"
passwd = "demotest123"
mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd
)

cursor = mydb.cursor()
cursor.execute("CREATE DATABASE IF NOT EXISTS inventory")
cursor.execute("show databases") 

for (databases) in cursor:
     print (databases[0])

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
bytearray(b'information_schema')
bytearray(b'inventory')
bytearray(b'mysql')
bytearray(b'performance_schema')


### Create the Table
Next we must create the table within the Database which will hold our inventory data.  We will call this table `product_table` which is defined by a _schema_ which contains the various columns along with datatypes. Our schema is: ` product_table (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2))`

In [102]:
! pip install mysql-connector

import mysql.connector
mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)

cursor = mydb.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS product_table (ProductName varchar(50),Description varchar(255),ProductSku bigint(12),QuantityInStock int(5),ReorderLevel int(5),WholeSaleCost decimal(8,2),RetailCost decimal(8,2));")

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


#### **Lab**: Try filling in the code below to list the tables that exist in the database.  The SQL we will use is `show tables`, see the above example on viewing the databases and modify it to show the tables.

In [34]:
! pip install mysql-connector

import mysql.connector
## Make sure the Host matches your RDS instance along with the username/password created earlier.
mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)

cursor = mydb.cursor()

##Fill in the code here to view the tables on our database

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Import the data from CSV into SQL Table

Now we will import our data from the provided `Lab1_inventory.csv` into our newly created table.  The code below iterates through the csv provided, and inserts every row into our table.


In [98]:
! pip install mysql-connector
import mysql.connector
import csv

mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)


cur = mydb.cursor()

csv_data = csv.reader(open('./data_files/Lab1_Inventory.csv'), delimiter=',')

# This removes the header
next(csv_data) 

# Process each row in the CSV
for row in csv_data:
        cur.execute ("INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES (%s, %s, %s, %s, %s, %s, %s)",row)
        print("Inserted Row")
mydb.commit()
print("Success!")

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Inserted Row
Inserted Row
Inserted Row
Inserted Row
Success!


After executing the above Python code your table is now populated!  

### View The Data in the Table

In [103]:
! pip install mysql-connector

import mysql.connector

mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)

cursor = mydb.cursor()
cursor.execute("SELECT * FROM product_table") 

result = cursor.fetchall()
print(result)


You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[('Mens Shampoo', 'Best brand Mens Shampoo', 1034403303, 10, 3, Decimal('3.25'), Decimal('6.99')), ('ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', 103443101, 4, 4, Decimal('8.00'), Decimal('12.99')), ('KeepFresh Detergent', 'Original detegent scent for washing clothes', 1034411101, 15, 3, Decimal('8.25'), Decimal('13.99')), ('MouthWash', 'Fresh Mint Mout wash', 122033020, 1, 5, Decimal('3.14'), Decimal('4.99')), ('Sample Product', 'Sample Description', 123456, 4, 2, Decimal('3.55'), Decimal('9.99'))]


#### **Lab:** Add your own data into the table!
Using the SQL Insert Statement add a couple rows into the table, and then display the result. Example: `INSERT INTO product_table (ProductName,Description,ProductSku,QuantityInStock,ReorderLevel,WholeSaleCost,RetailCost) VALUES ('Sample Product', 'Sample Description', 123456, 4, 2, 3.55, 9.99)`


In [105]:
! pip install mysql-connector
import mysql.connector
import csv

mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)

cursor = mydb.cursor()
## Add your own execute Insert statement here!    
 
mydb.commit()

## Prints out the rows in the table
cursor.execute("SELECT * FROM product_table") 

result = cursor.fetchall()
print(result)

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[('Mens Shampoo', 'Best brand Mens Shampoo', 1034403303, 10, 3, Decimal('3.25'), Decimal('6.99')), ('ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', 103443101, 4, 4, Decimal('8.00'), Decimal('12.99')), ('KeepFresh Detergent', 'Original detegent scent for washing clothes', 1034411101, 15, 3, Decimal('8.25'), Decimal('13.99')), ('MouthWash', 'Fresh Mint Mout wash', 122033020, 1, 5, Decimal('3.14'), Decimal('4.99')), ('Sample Product', 'Sample Description', 123456, 4, 2, Decimal('3.55'), Decimal('9.99'))]


### Querying Our Data

A query is a question we ask the database about our data. The language of retrieving our data is call _Structured Query Language_ or _SQL_.  
To ask our database a question we must do so in the format `SELECT * FROM product_table WHERE WholeSaleCost > 7` The field `WholeSaleCost` can be any colume in our database.  Check out [SQL Operators from W3](https://www.w3schools.com/sql/sql_operators.asp) to understand the questions we can ask about our data.

#### **Lab:** Try querying our database to find products that have a retail cost more than $6.

In [108]:
! pip install mysql-connector

import mysql.connector

mydb = mysql.connector.connect(
  host=host,
  user=user,
  passwd=passwd,
  database="inventory"
)

cursor = mydb.cursor()
## Fill out the Query to find products that have a retail cost greater than $6

result = cursor.fetchall()
print(result)

You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[('Mens Shampoo', 'Best brand Mens Shampoo', 1034403303, 10, 3, Decimal('3.25'), Decimal('6.99')), ('ScrewDriver Toolset', '3 Phillips, 2 Flathead toolset', 103443101, 4, 4, Decimal('8.00'), Decimal('12.99')), ('KeepFresh Detergent', 'Original detegent scent for washing clothes', 1034411101, 15, 3, Decimal('8.25'), Decimal('13.99')), ('Sample Product', 'Sample Description', 123456, 4, 2, Decimal('3.55'), Decimal('9.99'))]
